# Cell Segmentation Competition

## Evaluation
This evaluation part is highly based on the [SCS](https://doi.org/10.1038/s41592-023-01939-3) paper and the source code. We thank the authors for their great work.

This notebook will show you how to evaluate the result. The evaluation metric is from [SCS](https://doi.org/10.1038/s41592-023-01939-3):

We evaluate the performance of different methods by examining the transcriptional correlation between different regions of a segmentation mask. Specifically, for each nucleus detected using the staining, a target cell $c$ is assigned using our cell segmentation by selecting the cell with the largest overlap with the source nucleus. A similar approach is used to select $c^{\prime}$ based on the cell segmentation of compared method. Gene expression profiles considering all the genes for three regions are computed: (1) the intersection between $c$ and $c^{\prime}$, denoted $\mathbf{x}_{\text {int }}$, (2) the region covered by $c$ but not $c^{\prime}$, denoted $\mathbf{x}_c$, (3) and region covered by $c^{\prime}$ but not $c$, denoted as $\mathbf{x}_c^{\prime}$. Then the Pearson linear correlation is used to measure the similarity of gene expression between $\mathbf{x}_c$ and $\mathbf{x}_{\text {int }}$, as well as $\mathbf{x}_c^{\prime}$ and $\mathbf{x}_{\text {int }}$. The segmentation is said to be better if its difference region has a higher correlation with the intersection region.


When evaluating the accuracy of two cell segmentation using the correlation, a low correlation can result from one region having only a few detected RNAs aggregated. For example, one cell mask is almost contained in another cell mask, the difference region for the first cell mask will be very small. To make fair comparisons, we considered only cell mask pairs that all the three regions (the intersection region and two difference regions) have at least 100 RNA molecules aggregated. This cell filtering step can be found in the [supplementary note 3](https://static-content.springer.com/esm/art%3A10.1038%2Fs41592-023-01939-3/MediaObjects/41592_2023_1939_MOESM1_ESM.pdf) of the SCS paper.

For evaluation, the mapping from spot coordinates to cell indexes saved in a text file is needed.
Each line has the following format:

```row:column  cell_id```

where `row:column` is the coordinate of one spot indicating which row and column the spot is located in from the upper left corner, and `cell_id` is the index of the cell to which the spot belongs.

In most cases, your result is a 2D segmentation mask with different labels like the segementation result in `adata.layers['watershed_labels']` in [preprocess_demo.ipynb](preprocess_demo.ipynb). You can refer to the following code to postprocess your result to the standard format to evaluate and submit.

In [2]:
import os
import anndata as ad
# Load annotated data saved in h5ad format
startx = 0
starty = 3600
patch_size = 1200

adatasub = ad.read_h5ad(os.path.join('./results', f'watershed_labels_{startx}:{starty}:{patch_size}:{patch_size}.h5ad'))
nucl_labels = adatasub.layers['watershed_labels']

# Transfer it to the standard outputs format
fw = open(f'./results/spot2nucl_{startx}:{starty}:{patch_size}:{patch_size}.txt', 'w')
for i in range(nucl_labels.shape[0]):
    for j in range(nucl_labels.shape[1]):
        if nucl_labels[i, j] > 0:
            fw.write(str(i) + ':' + str(j) + '\t' + str(nucl_labels[i, j]) + '\n')
fw.close()

### Example

Here we provide an example of how to evaluate the result. The example result is produced by SCS, saved in `seg/spot2cell_SCS_6000/9600/1200/1200.txt`. You can evaluate your result by replacing it with your own result.

Run the following script for an example of comparing SCS segmentation with cellpose segmentation:
```
python evaluation.py gene/patch_tsv_6000:9600:1200:1200.tsv 0 0 1200 seg/spot2nucl_6000:9600:1200:1200.txt seg/spot2cell_SCS_6000:9600:1200:1200.txt seg/spot2cell_cellpose_6000:9600:1200:1200.txt
```
The script takes seven input:
- (*i*) gene counts of spots,
- (*ii*) row start index of the patch
- (*iii*) column start index of the patch
- (*iv*) patch size
- (*v*) nucleus segmentation, *i.e.*, mapping from spots to nuclei
- (*vi*) cell segmentation of method 1, *i.e.*, mapping from spots to cells、
- (*vii*) cell segmentation of method 2, *i.e.*, mapping from spots to cells.

The Pearson correlation statistics will be printed, and a boxplot summarizing the correlations will be saved in the `results` folder.


The output of the script contains:
- Number of total gene numbers
- Mean/Median of the Pearson correlation of the intersection region and the difference region of the two cell segmentations.
- Krukal-Wallis test p-value of the Pearson correlation of the intersection region and the difference region of the two cell segmentations.
- a boxplot showing the correlation of different methods. The x-axis is the method name, and the y-axis is the correlation. The correlation is calculated for each patch, and the boxplot shows the distribution of the correlation.

Output example:
```
python evaluation.py gene/patch_tsv_6000:9600:1200:1200.tsv 0 0 1200 seg/spot2nucl_6000:9600:1200:1200.txt seg/spot2cell_SCS_6000:9600:1200:1200.txt seg/spot2cell_cellpose_6000:9600:1200:1200.txt

total gene number: 18520
mean corr_seg1: 0.5299815426873342 mean corr_seg2: 0.32186504433685376 median corr_seg1: 0.525147860930285 median corr_seg2: 0.2910778526066163
Kruskal-Wallis H-test p-value: 9.272615211996793e-23, Sample num: 239
```
with a boxplot stored in `results` folder.

Your final score will be the average of the Pearson correlation of all patches and of different published methods.